# Linear Classifier for Clear-Cell Renal Carcinoma Grade

### Background:
Clear-cell renal cell carcinoma (ccRCC) is the most common subtype of renal cancers, and accounts for nearly 50,000 new cases annually. The Cancer Genome Atlas (TCGA) project profiled over 400 sporadic ccRCC tumors, and published dozens of clinical and molecular features describing these tumors. My lab has studied 40 tumors from heritable cases of ccRCC. Individuals with heritable ccRCC have a germline loss-of-function mutation in the gene VHL, and make up nearly 2% of all ccRCC cases. We have observed differences in the number of somatic genetic driver events within high and low Fuhrman grade tumors. The Fuhrman grading system is assigned by pathologist based on the shape and size of the nucleus of tumor cells, and classifies tumors into one of four possible grades. High Fuhrman grade ccRCC tumors have an increased ability for metastasis and lower survival rate. There is very limited information regarding the clinical and molecular features of high and low Fuhrman grade ccRCCs. The goal of my project will be to build a classifier for Fuhrman grade in ccRCCs.

### Summary of Features 
| **ID** | **Type** | **Description** |
|:-------------:|:-------------:|:-----:|
| Grade | Binary | Class variable. 0 = low Fuhrman grade, 1 = high Fuhrman grade |
| Gender | Binary | Male = 0, Female = 1 |
| Age | Discrete | Age at diagnosis |
| MaxDim | Continuous | Maximum tumor dimension, surrogate for tumor size |
| Mets | Binary | Presence of metastasis = 1, absence of metastasis = 0 |
| VHL | Binary | Presence of somatic mutation in VHL =1, absence = 0 |
| Stage | Discrete | Tumor stages. 0 = stage 1, 1 = stage 2, 2 = stage 3, 3 = stage 4. |
| *TotalMut* | Discrete | Total number of somatic exonic mutations |
| *TotalNonSyn* | Discrete | Total number of somatic exonic nonsynonymous mutations |
| *TotalSNVDriver* | Discrete | Total number of somatic exonic nonsynonymous mutations in known cancer driver genes |
| *ClonSNVDriver* | Discrete | Total number of somatic exonic nonsynonymous clonal driver mutations in know cancer driver genes |
| *SubclonSNVDriver* | Discrete | Total number of somatic exonic nonsynonymous subclonal driver mutations in know cancer driver genes |
| *DriverSNVRatio* | Continuous | Number of driver mutations / total number of exonic nonsynonymous mutations |
| *ClonSNVRatio* | Continuous | Number of clonal driver mutations / total number of driver mutations |

##### Low Grade = 196 cases
##### High Grade = 249 cases

### Methods:
Italics denotes features I computed for this project, and all other features are publicly available from TCGA. I will apply linear discriminant analysis on my feature set, and determine which (if any) features are linearly separable for each model. I will implement both learning algorithms from scratch. I will use leave-one-out cross-validation to determine how well each model generalizes. 



### Functions for calculating italicized features

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Oct 20 08:13:58 2015

@author: mitcheas@ohsum01.ohsu.edu

Calculates the cancer cell fraction for variants in a given VCF

Usage: CCF.py -vcf <input.vcf> -seg <input.seg> -pur <0.75> -o
Required inputs: 
1. vcf - VCF file, tumor read counts must be listed in the second to last column
2. seg  -  SEG file containing copy number ratio in log base 2 
3. pur - a value between 0 and 1 representing tumor purity
4. o - path to your output file

Output file will be written to the filename provided at input
The output file is a tab delimited file with the following headers:

CHROM	POS	READ_DEPTH	CCF	ERROR	CLONAL

CHROM - the chromosome (ie 1, 2, ..., X)
POS - nucleotide position
READ_DEPTH - number of reads mapped to nucleotide position
CCF - cancer cell fraction (usually a value between 0 and 1, values above 1 are likely and
		represent errors in purity, allele frequency, and integer copy number estimation)
ERROR - +/- error for the 95% confidence interval
CLONAL - binary T/F where T denotes clonal SNV and F denotes a subclonal SNV
"""

import math
import numpy as np
import sys
import string, re
from scipy import stats
import argparse

np.seterr(divide='ignore', invalid='ignore')

nonsyn_mut = ['frameshift_variant', 'missense_variant', 'splice_acceptor_variant',
				'splice_region_variant', 'stop_gained', 'inframe_deletion']

# FUNCTION: READ_FILE(FILENAME)
# Opens and Reads all lines of an input file 
def read_file(Filename):
	Filename = open(Filename, 'U')
	alllines = Filename.readlines()
	outlist = []
	for line in alllines:
		line = line.rstrip()
		line = string.split(line, '\t')
		outlist.append(line)
	return outlist
    
# FUNCTION: READ_FILE_hd(FILENAME)
# Opens and Reads all lines of an input file with a header
def read_file_hd(Filename):
	Filename = open(Filename, 'U')
	alllines = Filename.readlines()
	alllines = alllines[1:len(alllines)]
	outlist = []
	for line in alllines:
		line = line.rstrip()
		line = string.split(line, '\t')
		outlist.append(line)
	return outlist

# FUNCTION: READ_VCF(FILENAME)
# Reads all lines of an input VCF file
# Removes header lines of VCF
def read_vcf(Filename):
	Filename = open(Filename, 'U')
	alllines = Filename.readlines()
	outlist = []
	for line in alllines:
		if re.match('##SAMPLE=<ID=TUMOR', line):
			tumor = string.split(line, ',')[1]
			tumor = string.split(tumor, '=')[1]
			tumor = string.join((string.split(tumor, '-')[0:3]), '-')
		elif not re.match('#', line):
			line = line.rstrip()
			line = string.split(line, '\t')
			outlist.append(line)
	return tumor, outlist

# FUNCTION: CCF(VAF, Purity, Ploidy, CN)
# Calculate the cancer cell fraction
def CCF(mreads, wtreads, pur, cnT, cnN):
	treads = float(mreads) + float(wtreads)
	vaf = float(mreads)/float(treads)
	cnT = float(cnT)  
	n_obs = float(vaf/pur) * ((pur * cnT) + (cnN * (1-pur)))
	n_exp = Nexp(int(cnT))
	x = n_obs/n_exp
	return x

# FUNCTION: Nexp(CN):
# CN = integer copy number returned from COPY_NUM function
# Calculate and return the expected mutation copy number, n
# Expected mutation copy number is estimated by determining the maximum likelihood
# of the copy number state given the observed copy number and the assumption that
# a somatic mutation is most likely to be heterozygous.
#
# To calculate Nexp without the assumption that a somatic mutation is heterozygous (ie in 
# the case of mutations residing in oncogenes), you can edit this function to also require
# the VAF (variant allele frequency) of a mutation.
def Nexp(cn):
	success = np.zeros(cn)
	total = np.zeros(cn)
	cn_range = np.arange(1, cn+1)
	for i in range(1,cn+1):
		x = float(math.factorial(cn))
		x = x/(float(math.factorial(i))*float(math.factorial(cn-i)))
		total[i-1] = x
	success = total * cn_range
	n = sum(success)/ sum(total)
	return n

#FUNCTION: bootstrap_CCF(mreads, wtreads)
# Bootstrap resampling of the mutant (mreads) and wild-type (wtreads) reads to create a 
# distribution of possible mutant/wild-type read populations. Determines CCF for every
# bootstrap sample. Calculates and returns the error (95% confidence interval of the ccf)
def Bootstrap_CCF(mreads, wtreads, pur, cnT, cnN):
	I = 10000
	ccf = []
	for x in range(I):
		m, wt = Resample(mreads, wtreads)
		c = CCF(m, wt, pur, cnT, cnN)
		ccf.append(c)
	ci = ConfInv(ccf)
	return ci
		
# Calculate and return the error based on a 95% confidence interval
def ConfInv(ccf):
	n, min_max, mean, var, skew, kurt = stats.describe(ccf)
	s = math.sqrt(var)
	c = float(s) * float(1.96)
	return c

# Return a count for mutant and wild-type reads, from a random resample. 
def Resample(mreads, wtreads):
	reads = np.concatenate((np.ones(mreads), np.zeros(wtreads)))
	x = np.random.choice(reads, (mreads+wtreads), replace="TRUE")
	m = sum(x)
	wt = len(x) - m
	return m, wt
    
# FUNCTION:  COPYNUM(seg_dict, chrm, pos, cnN)
# Calculate and return the integer copy number from a SEG file for a nucleotide position 
# on a given chromosome, where the average tumor copy number ratio is provided in log base 
# 2 format. Seg_dict is a dictionary containing all log base 2 copy number ratios read from
# the seg file.
def Copy_Num(seg_dict, chrm, pos, cnN):
    d = []
    if chrm not in seg_dict.keys():
		c = 2.0
		return c
    elif pos < sorted(seg_dict[chrm].keys())[0]:
		c = 2.0
		return c
    else:
        for start in sorted(seg_dict[chrm].keys()):
            if pos >= start:
            	d.append(start)
        for start in d:
			end = seg_dict[chrm][start][0]
			if pos <= (end+1):
				cn = seg_dict[chrm][start][1]
				c1 = pow(2, cn)
				c = c1 * cnN
				c = round(c)
				return c
			else:
				c = 2.0
    return c


### Main code for calculating features

In [ ]:
def main():
	
	FEATURES = {}
	TMP = read_file_hd('KIRC_features.txt')
	for line in TMP:
		t_id = line[0]
		grade = line[1]
		gender = line[2]
		age = line[3]
		maxd = line[4]
		mets = =line[5]
		stage = line[6]
		if t_id not in FEATURES.keys():
			FEATURES[t_id] = []
		FEATURES[t_id] = [grade, gender, age, mxd, mets, stage]
		
	BIC_SEG = {} ## dictionary for copy number
	TMP = read_file_hd('KIRC_edited.seg')
	for line in TMP:
		t_id = string.join((string.split(line[0], '-')[0:3]), '-')
		chr = line[1]
		st = int(line[2])
		end = int(line[3])
		seg = float(line[5])
		if t_id not in BIC_SEG.keys():
			BIC_SEG[t_id] = {}
		if chr not in BIC_SEG[t_id].keys():
			BIC_SEG[t_id][chr] = {}
		if st not in BIC_SEG[t_id][chr].keys():
			BIC_SEG[t_id][chr][st] = []
		BIC_SEG[t_id][chr][st] = [end, seg]
	
	DRIVERS = read_file('drivers.txt')
	
	## Loop through all VCF files	
	infiles = read_file('vcffiles.txt')
	
	for fi in infiles:
		# Open and read input files
		tum_id, vcf = read_vcf(fi)		
		
		# Check that tumor ID is in features list and BIC_SEG
		if tum_id in FEATURES.keys() and tum_id in BIC_SEG.keys():
			gender = FEATURES[tum_id][1]
			# Write header to output file
			# outfile.write('CHROM\tPOS\tREAD_DEPTH\tCCF\tERROR\tCLONAL\n')

			tmp = [0, 0, 0, 0, 0, 0, 0]
			VHL = 0
			# 0:'TotalMut', 1: 'TotalNonSyn', 2: 'TotalSNVDriver', 3:'ClonSNVDriver',
			# 4: 'SubClonSNVDriver', 5: 'DriverSNVRatio', 6: 'ClonSNVRatio'
			
			## Calculate CCF for each variant and write to outfile
			for line in vcf:
				chrm = line[0]
				chrm = string.split(chrm, 'chr')[1] # removes the 'chr' prefix, if present
				pos = int(line[1])
				filt = line[6]
				mut_type = string.split(line[7], '|')[1]
	
				if filt == 'PASS':
					tmp[0] += 1
				if filt == 'PASS' and mut_type in nonsyn_mut:
					tmp[1] += 1
					# get mutant and wild-type allele read count for the tumor
					# tumor should be last column in VCF
					mreads = float(string.split((string.split(line[-1],':')[1]),',')[1])
					wtreads = float(string.split((string.split(line[-1],':')[1]),',')[0])
					depth = int(mreads + wtreads)
					gene = string.split(line[7], '|')[3]
					if gene == 'VHL':
						VHL = 1
					if gene in DRIVERS:
						tmp[2] += 1
						# Determine chromosomal copy number for normal cells
						# Autosome copy number = 2
						# X copy number = 1 if male, or X = 2 if female
						# Y copy number = 1
						if chrm == 'X':
							if gender == 0:
								cnN = 1.0
							elif 'Y' in bic_seg.keys():
								cnN = 1.0
							else:
								cnN = 2.0
						elif chrm == 'Y':
							cnN = 1.0
						else:
							cnN = 2.0
						cnT = Copy_Num(bic_seg, chrm, pos, cnN)
						ccf = CCF(mreads, wtreads, args.pur, cnT, cnN)
						err = Bootstrap_CCF(mreads, wtreads, args.pur, cnT, cnN)
						if (ccf + err) >= 1.0: #clonal
							tmp[3] += 1
						else: #subclona
							tmp[4] += 1
			tmp[5] = float(tmp[2]/tmp[1])
			tmp[6] = float(tmp[3]/tmp[2])
			for x in tmp:
				FEATURES[t_id].append(x)
			outfile.write('%s' % (t_id))
			outfile.write('\t'.join(FEATURES[t_id]))
			outfile.write('\n')				
	#outfile.write('%s\t%d\t%i\t%f\t%f\t%c\n' % (chrm, pos, depth, ccf, err, clonal))
	outfile.close()

if __name__ == "__main__":
	main()


### Functions for Implementing Linear Discriminate Analysis (LDA)

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 14 08:36:08 2016

@author: mitcheas@ohsum01.ohsu.edu
"""

import numpy as np
import pandas
import seaborn as sns
import random
#import math
#import os, sys, string
#import matplotlib.pyplot as plt

# LOOCV
def LOOCV(X, y):
    rows = X.shape[0]
    cols = X.shape[1]
    E = np.matrix(np.zeros(rows))

def TrainLDA(X, y):
    # Compute class mean
    mu_class = MeanVector(X, y) 
    
    # Compute within class scatter matrix
    S_w = WithinClScatter(X, y, mu_class)
    
    # Compute between class scatter matrix
    S_b = BtwnClScatter(X, y, mu_class)
    
    # Solve eigenvectors
    eig_vals, eig_vecs = np.linalg.eig((np.linalg.inv(S_w) * S_b))
    print eig_vecs.shape
    
    # Get new feature subspace
    W = subspace(eig_vals, eig_vecs)
    
    return W

    
def WithinClScatter(X, y, muvec):
    cols = X.shape[1]
    numcl = muvec.shape[0]
    S = np.zeros((cols, cols))
    
    for cl, mu in zip(range(0, numcl), muvec):        
        X_cl = X[np.where(y == cl)]
        w = np.sum((X_cl - mu), axis=0)
        w = w.reshape(cols,1)
        S += w * w.T       
    return S

def BtwnClScatter(X, y, muvec):
    cols = X.shape[1]
    numcl = muvec.shape[0]
    S = np.zeros((cols, cols))
    
    # Compute overall mean
    mu_all = np.mean(X, axis=0) 
    
    for cl, mu in zip(range(0, numcl), muvec):
        N = X[np.where(y==cl)].shape[0]
        mu = mu.reshape(cols, 1)        
        mu_all = mu_all.reshape(cols, 1)
        S += N * ((mu - mu_all) * (mu - mu_all).T)
    return S

# Computes and returns the class mean for each feature 
# X = feature matrix
# y = class vector
def MeanVector(X, y):
    cols = X.shape[1]
    c = np.unique(y)
    numcl = c.shape[0]
    muvec = np.zeros((numcl, cols))
    for cl in c:
        muvec[cl] = np.mean(X[np.where(y == cl)], axis=0)
    return muvec

def subspace(eig_vals, eig_vecs, n=2):
    cols = eig_vecs.shape[1]
    W = np.zeros((n, cols))
    
    eig_pairs = []
    for i in range(0, cols):
        eig_pairs.append((np.abs(eig_vals[i]), eig_vecs[:,i]))
    
    # Sort eigenvectors by eigenvalues in descending order
    eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)
    for i in eig_pairs:
        print i
   
    # Select n number of eigenvectors and store in W   
    for i in range(0, n):
        W[i] = eig_pairs[i][1]
    return W

def ProjectLDA(X, W):
    rows = X.shape[0]
    cols = X.shape[1]
    k = W.shape[0]
    Y_lda = np.zeros((rows, k))

    # Project observations onto new feature subspace
    for i in range(0, k):
        print W[i].shape 
        print X.shape
        Y_lda[:,i] = X.dot(W[i])
    return Y_lda

# c = number of classes       
def Classify(X, pri, mu_class, W):
    c = mu_class.shape[0]
    rows = X.shape[0]
    cols = X.shape[1]
    f = np.zeros((rows, c))
    pred = np.zeros((rows))
    
    # Discriminant function
    for cl in range(0, c):
        mu = mu_class[cl].reshape(1, cols)
        w = W[cl].reshape(1, cols)
        a = (mu * w).dot(X.T)
        b = 0.5 * ((w * mu).dot(mu.T))
        c = np.log2(pri[cl])
        f[:, cl] = a - b + c
        
    # Choose class based on largest discriminant
    for i, obs in enumerate(X):
        q = f[i]
        cl = np.where(q == q.max())
        pred[i] = cl[0]
        
    return pred
     
def Priors(y):
    c = np.unique(y)
    numcl = c.shape[0]
    n = float(y.shape[0]) 
    p = np.zeros(numcl)
    
    for i in c:
        p[i] = float(len((np.where(y == i))[0]) / n)
    return p
    
# FUNCTION: ConfMtrx(Y, y)
# Calculates the mean squared error of the actual target value (y)
# and the predicted target value 
# y_pred is an array of predicted y values
# y_obs is an array of actual class values 
def ConfMtrx(pred, obs):
    err = np.zeros((2,2))
    rows = pred.shape[0]
    i = 0
    while (i < rows):
        if obs[i] == pred[i]:
            if obs[i] == 1.0:
                err[0,0] += 1
            else:
                 err[1,1] += 1
        else:
            if obs[i] == 1.0:
                err[0,1] += 1
            else:
                 err[1,0] += 1
        i += 1
    return err
    
def MissClassErr(err):
    total = float(np.sum((np.sum(err, axis=0)), axis=0))
    tp = float(np.trace(err))
    fp = float(np.trace(np.fliplr(err)))
    e = fp/total
    a = tp/total
    return e, a

### Implementing LDA

In [ ]:
headers = ['GRADE','GENDER','AGE','MAXDIM','METS','VHL','STAGE','TOTALMUT',
           'TOTALNONSYN','TOTALDRIVER','CLONDRIVER','SUBCLONDRIVER',
           'DRIVRATIO','CLONRATIO']
#F = (3,4,6,7,9,12,13)
#F = (1,2,3,4,6,10) ## Slected Features
F = (1,2,3,4,6,7,8,10,11,12,13) ## All Features
np.random.seed(1121)

alldata = np.loadtxt('features_KIRC.txt')

alldata = random.sample(alldata, len(alldata))
alldata = np.asarray(alldata)
Xdata = alldata[:,F]
#Xdata = alldata[:,(1,2,3,4,6,10)] 
Ydata = alldata[:,0]

#alldata = np.loadtxt('Iris/iris.data', delimiter=',')
#t = np.where(alldata[:,-1] != 2)
#twoclass = alldata[t]
#Xdata = twoclass[:,(0,1,2,3)]
#Ydata = twoclass[:,(4)]

R = 208
Xtrain = Xdata[0:R,]
Xtest = Xdata[R:,]
Ytrain = Ydata[0:R,]
Ytest = Ydata[R:,]

Wtrain = TrainLDA(Xtrain, Ytrain)
KIRC_lda = ProjectLDA(Xtrain, Wtrain)
pri = Priors(Ytrain)
mu_class = MeanVector(Xtrain, Ytrain)
pred = Classify(Xtest, pri, mu_class, Wtrain)

err = ConfMtrx(pred, Ytest)
e, a = MissClassErr(err)
print "Error: ", e, " Accuracy: ", a

Y = Ytrain.reshape(R,1)
a = np.concatenate((KIRC_lda, Y), axis=1)
al = pandas.DataFrame(data=a, columns=('x','y','c'))
sns.lmplot('x', 'y', data=al, fit_reg=False, scatter_kws={"marker": "D", "s": 100}, hue='c')
plt.show()

plt.hist(al.loc[al['c'] == 0]['x'], bins=10, color='b', label='0', alpha=0.7)
#plt.hold(True)
plt.hist(al.loc[al['c'] == 1]['x'], bins=10, color='g', label='1', alpha=0.7)
plt.legend(loc='upper right')
plt.show()
for i in F:
    print headers[i]
